# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 3:05PM,241450,10,0085850943,ISDIN Corporation,Released
1,Jul 8 2022 3:05PM,241450,10,0085850950,ISDIN Corporation,Released
2,Jul 8 2022 3:05PM,241450,10,0085850962,ISDIN Corporation,Released
3,Jul 8 2022 3:05PM,241450,10,0085850965,ISDIN Corporation,Released
4,Jul 8 2022 3:05PM,241450,10,0085850964,ISDIN Corporation,Released
5,Jul 8 2022 3:05PM,241450,10,0085850970,ISDIN Corporation,Released
6,Jul 8 2022 3:05PM,241450,10,0085850946,ISDIN Corporation,Released
7,Jul 8 2022 3:05PM,241450,10,0085850974,ISDIN Corporation,Released
8,Jul 8 2022 3:05PM,241450,10,0085850961,ISDIN Corporation,Released
9,Jul 8 2022 3:05PM,241450,10,0085850987,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
58,241447,Released,12
59,241449,Released,46
60,241450,Released,28
61,241451,Released,1
62,241452,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241447,NaN,NaN,12.0
241449,NaN,NaN,46.0
241450,NaN,NaN,28.0
241451,NaN,NaN,1.0
241452,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241360,0.0,0.0,2.0
241361,0.0,0.0,1.0
241362,0.0,0.0,17.0
241366,24.0,1.0,0.0
241368,0.0,1.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241360,0,0,2
241361,0,0,1
241362,0,0,17
241366,24,1,0
241368,0,1,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241360,0,0,2
1,241361,0,0,1
2,241362,0,0,17
3,241366,24,1,0
4,241368,0,1,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241360,,,2
1,241361,,,1
2,241362,,,17
3,241366,24,1,
4,241368,,1,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation
28,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation
74,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
76,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc."
83,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC"
84,Jul 8 2022 2:54PM,241447,10,ISDIN Corporation
96,Jul 8 2022 2:40PM,241446,10,Emerginnova
97,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc."
98,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc."
99,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,,,28
1,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,,,46
2,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
3,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc.",,,7
4,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC",,,1
5,Jul 8 2022 2:54PM,241447,10,ISDIN Corporation,,,12
6,Jul 8 2022 2:40PM,241446,10,Emerginnova,,,1
7,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",,,1
8,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",,,1
9,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,28,,
1,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,46,,
2,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
3,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc.",7,,
4,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC",1,,
5,Jul 8 2022 2:54PM,241447,10,ISDIN Corporation,12,,
6,Jul 8 2022 2:40PM,241446,10,Emerginnova,1,,
7,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",1,,
8,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",1,,
9,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,28,,
1,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,46,,
2,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
3,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc.",7,,
4,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,28.0,NaN,NaN
1,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,46.0,NaN,NaN
2,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN,NaN
3,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc.",7.0,NaN,NaN
4,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,28.0,0.0,0.0
1,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,46.0,0.0,0.0
2,Jul 8 2022 3:03PM,241452,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0,0.0
3,Jul 8 2022 3:02PM,241433,20,"ACI Healthcare USA, Inc.",7.0,0.0,0.0
4,Jul 8 2022 3:02PM,241451,16,"SHL Pharma, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8207725,266.0,6.0,2.0
102,482848,2.0,0.0,0.0
15,241378,124.0,8.0,36.0
16,965760,14.0,5.0,0.0
18,241422,1.0,0.0,0.0
19,965675,3.0,7.0,29.0
20,724207,27.0,5.0,3.0
22,724326,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8207725,266.0,6.0,2.0
1,102,482848,2.0,0.0,0.0
2,15,241378,124.0,8.0,36.0
3,16,965760,14.0,5.0,0.0
4,18,241422,1.0,0.0,0.0
5,19,965675,3.0,7.0,29.0
6,20,724207,27.0,5.0,3.0
7,22,724326,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,266.0,6.0,2.0
1,102,2.0,0.0,0.0
2,15,124.0,8.0,36.0
3,16,14.0,5.0,0.0
4,18,1.0,0.0,0.0
5,19,3.0,7.0,29.0
6,20,27.0,5.0,3.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,266.0
1,102,Released,2.0
2,15,Released,124.0
3,16,Released,14.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,22
Status,,,,,,,,
Completed,2.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0
Executing,6.0,0.0,8.0,5.0,0.0,7.0,5.0,0.0
Released,266.0,2.0,124.0,14.0,1.0,3.0,27.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,22
0,Completed,2.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0
1,Executing,6.0,0.0,8.0,5.0,0.0,7.0,5.0,0.0
2,Released,266.0,2.0,124.0,14.0,1.0,3.0,27.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,22
0,Completed,2.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0
1,Executing,6.0,0.0,8.0,5.0,0.0,7.0,5.0,0.0
2,Released,266.0,2.0,124.0,14.0,1.0,3.0,27.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()